In [81]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from random import randint
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## Obtaining the web page

In [19]:
url = 'https://www.billboard.com/charts/hot-100'

In [20]:
response = requests.get(url)
response.status_code

200

In [21]:
soup = BeautifulSoup(response.content, "html.parser")

## Creating the dataframe

In [22]:
titles = []
for title in soup.select("h3")[4:104]:
    titles.append(title.get_text().strip())

In [23]:
artists = []
for artist in soup.select("li.o-chart-results-list__item.\/\/.lrv-u-flex-grow-1.lrv-u-flex.lrv-u-flex-direction-column.lrv-u-justify-content-center.lrv-u-border-b-1.u-border-b-0\@mobile-max.lrv-u-border-color-grey-light.lrv-u-padding-l-1\@mobile-max > span"):
    artists.append(artist.get_text().strip())

In [30]:
#artists

In [31]:
top_100 = pd.DataFrame({"artist": artists, "song": titles})
top_100

,artist,song
0,Taylor Swift,All Too Well (Taylor's Version)
1,Adele,Easy On Me
2,The Kid LAROI & Justin Bieber,Stay
3,Lil Nas X & Jack Harlow,Industry Baby
4,Silk Sonic (Bruno Mars & Anderson .Paak),Smokin Out The Window
...,...,...
95,Swedish House Mafia & The Weeknd,Moth To A Flame
96,Cody Johnson,'Til You Can't
97,Doja Cat,Woman
98,The Weeknd,Take My Breath


## Obtaining the second list

In [32]:
url1 = 'https://www.elportaldemusica.es/lists/top-100-canciones/2021/45'

In [33]:
response1 = requests.get(url1)
response1.status_code

200

In [34]:
soup1 = BeautifulSoup(response1.content, "html.parser")

In [35]:
#soup1

In [36]:
artists1 = []
for name in soup1.select("div.related"):
    artists1.append(name.get_text().strip())

In [37]:
songs1 = []
for name in soup1.select("div.name"):
    songs1.append(name.get_text().strip())

In [38]:
second_top = pd.DataFrame({"artist": artists1, "song": songs1})
second_top

,artist,song
0,"C. Tangana, Nathy Peluso",Ateo
1,"zzoilo, Aitana",Mon Amour (remix)
2,BIZARRAP / ANUEL AA,ANUEL AA: BZRP MUSIC SESSION: VOL. 46
3,MYKE TOWERS,EXPERIMENTO
4,"Rvssian, Rauw Alejandro & Chris Brown",Nostálgico
...,...,...
95,KAROL G,SEJODIOTO
96,Olivia Rodrigo,Good 4 u
97,BIZARRAP / NICKY JAM,"NICKY JAM: BZRP MUSIC SESSIONS, VOL. 41"
98,"Jhay Cortez, Skrillex",En Mi Cuarto


### Asking a song name

In [39]:
def recommending_song(song):
    songs_to_recommend_top_100 = []
    position_top_100 = []
    songs_to_recommend_second_top = []
    position_second_top = []
    for i in range(100):
        if song == top_100['song'][i]:
            songs_to_recommend_top_100.append(top_100['song'][i])
            position_top_100.append(i)
        elif song == second_top['song'][i]:
            songs_to_recommend_second_top.append(second_top['song'][i])
            position_second_top.append(i)
    if len(songs_to_recommend_top_100)> 0:
        random_number = position_top_100[0]
        while random_number == position_top_100[0]:
            random_number = randint(0,99)
        i_recommend = top_100['song'][random_number]
        return print("Good choice! This song is now in the position number " + str(position_top_100[0]+1) + " of our first top from " + top_100['artist'][position_top_100[0]] + ". Here we have another recommendation of this top 100 songs: " + i_recommend + " from " + top_100['artist'][random_number])
    elif len(songs_to_recommend_second_top)> 0:
        random_number = position_second_top[0]
        while random_number == position_second_top[0]:
            random_number = randint(0,99)
        i_recommend = second_top['song'][random_number]
        return print("Good choice! This song is now in the position number " + str(position_second_top[0]+1) + " of our second top from " + second_top['artist'][position_second_top[0]] + ". Here we have another recommendation of this top 100 songs: " + i_recommend + " from " + second_top['artist'][random_number])
    else:
        print("Sorry, now we can't help you with that song, but if you want to try again with another one...")
        return 0

In [42]:
def recommending_artist(artist):
    songs_to_recommend_top_100 = []
    position_top_100 = []
    songs_to_recommend_second_top = []
    position_second_top = []
    for i in range(100):
        text1 = top_100['artist'][i]
        text2 = second_top['artist'][i]
        pattern = artist
        number_1 = re.findall(pattern, text1)
        number_2 = re.findall(pattern, text2)
        if len(number_1)> 0:
            songs_to_recommend_top_100.append(top_100['song'][i])
            position_top_100.append(i)
            number_1.pop(0)
        if len(number_2)> 0:
            songs_to_recommend_second_top.append(second_top['song'][i])
            position_second_top.append(i)
            number_2.pop(0)
    if len(songs_to_recommend_top_100)> 0:
        print('Good choice! This artist has ' + str(len(songs_to_recommend_top_100)) + ' songs:')
        for i in range(len(songs_to_recommend_top_100)):
            print('-' + songs_to_recommend_top_100[i] + ' in position ' + str(position_top_100[i] + 1))
        return 
    elif len(songs_to_recommend_second_top)> 0:
        print('Good choice! This artist has ' + str(len(songs_to_recommend_second_top)) + ' songs:')
        for i in range(len(songs_to_recommend_second_top)):
            print('-' + songs_to_recommend_second_top[i] + ' in position ' + str(position_second_top[i] + 1))
        return 
    else:
        print("Sorry, now we can't help you with that artist, but if you want to try again with another one...")
        return 0

In [45]:
def asking_song():
    print("Which song do you want?")
    song_name = input()
    value = recommending_song(song_name)
    while value == 0:
        print("Which new song do you want?")
        song_name = input()
        value = recommending_song(song_name)
    return

In [46]:
def asking_artist():
    print("Which artist do you want?")
    artist_name = input()
    value = recommending_artist(artist_name)
    while value == 0:
        print("Which new artist do you want?")
        artist_name = input()
        value = recommending_artist(song_name)
    return

In [ ]:
asking_song()

In [ ]:
asking_artist()

In [78]:
def asking():
    print("If you want to search by song enter 1, and if you want to search by artist enter 2")
    search = input()
    while (search != '1') & (search != '2'):
        print(type(search))
        print("This isn't a valid value... Try again")
        print("If you want to search by song enter 1, and if you want to search by artist enter 2")
        search = input()
    if search == '1':
        return asking_song()
    elif search == '2':
        return asking_artist()

In [80]:
asking()

If you want to search by song enter 1, and if you want to search by artist enter 2


 Sabe 


<class 'str'>
This isn't a valid value... Try again
If you want to search by song enter 1, and if you want to search by artist enter 2


 Sabe


<class 'str'>
This isn't a valid value... Try again
If you want to search by song enter 1, and if you want to search by artist enter 2


...

Which song do you want?


 Sabe 


Sorry, now we can't help you with that song, but if you want to try again with another one...
Which new song do you want?


 Sabe


Good choice! This song is now in the position number 32 of our second top from Nicki Nicole & Rauw Alejandro. Here we have another recommendation of this top 100 songs: Ateo from C. Tangana, Nathy Peluso
